In [1]:
import sys
import nltk
from textblob import TextBlob
from newspaper import Article
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QLabel, QPushButton, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QFont, QColor, QLinearGradient, QPalette, QBrush, QClipboard
from PyQt5.QtCore import Qt

# Example training data
data = {
    'text': ['Politics news about elections', 'Latest tech trends in 2022', 'Sports update: Football world cup', 'Tech startups booming', 'Election results are out'],
    'category': ['Politics', 'Technology', 'Sports', 'Technology', 'Politics']
}
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['category'], test_size=0.2, random_state=42)

# Train a Naive Bayes classifier
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('AI-based Summarizer')
        self.setGeometry(100, 100, 1200, 700)

        font = QFont('Arial', 10)
        self.setFont(font)

        gradient = QLinearGradient(0, 0, 0, 400)
        gradient.setColorAt(0.0, QColor(240, 240, 240))
        gradient.setColorAt(1.0, QColor(180, 180, 180))

        palette = QPalette()
        palette.setBrush(QPalette.Window, QBrush(gradient))
        self.setPalette(palette)

        self.layout = QVBoxLayout()

        self.labels = ["Title", "Author", "Publication date", "Summary", "Sentiment Analysis", "Category", "URL"]
        self.textboxes = {}

        for label in self.labels:
            row_layout = QHBoxLayout()

            self.label = QLabel(label)
            self.label.setStyleSheet('font-weight: bold; color: #333333;')
            row_layout.addWidget(self.label)

            self.textboxes[label] = QTextEdit(self)
            self.textboxes[label].setFixedHeight(30 if label != "Summary" else 360)
            if label == "URL":
                self.textboxes[label].setReadOnly(False)  # Allows input for URL
            else:
                self.textboxes[label].setReadOnly(True)
            self.textboxes[label].setStyleSheet('background-color: #ffffff; border-radius: 8px; padding: 5px;')
            row_layout.addWidget(self.textboxes[label])

            if label != "URL":
                copy_button = QPushButton('Copy', self)
                copy_button.clicked.connect(lambda _, key=label: self.copy_to_clipboard(key))
                copy_button.setStyleSheet('background-color: #5CACEE; font-weight: bold; color: white; border-radius: 8px; padding: 10px;')
                row_layout.addWidget(copy_button)

            self.layout.addLayout(row_layout)

        summarize_button = QPushButton('Summarize', self)
        summarize_button.setStyleSheet('background-color: #4CAF50; font-weight: bold; color: white; border-radius: 8px; padding: 10px;')
        summarize_button.clicked.connect(self.summarize)
        self.layout.addWidget(summarize_button)

        exit_button = QPushButton('Exit', self)
        exit_button.setStyleSheet('background-color: #FF6347; font-weight: bold; color: white; border-radius: 8px; padding: 10px;')
        exit_button.clicked.connect(self.close)
        self.layout.addWidget(exit_button)

        self.central_widget = QWidget()
        self.central_widget.setLayout(self.layout)
        self.setCentralWidget(self.central_widget)

    def copy_to_clipboard(self, key):
        clipboard = QApplication.clipboard()
        clipboard.setText(self.textboxes[key].toPlainText())

    def summarize(self):
        url = self.textboxes["URL"].toPlainText().strip()
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()

        self.textboxes["Title"].setText(article.title)
        self.textboxes["Author"].setText(', '.join(article.authors))
        self.textboxes["Publication date"].setText(article.publish_date.strftime('%Y-%m-%d %H:%M:%S') if article.publish_date else 'Not available')
        self.textboxes["Summary"].setText(article.summary)

        analysis = TextBlob(article.text)
        polarity = analysis.polarity
        sentiment = "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"
        self.textboxes["Sentiment Analysis"].setText(f'Polarity: {polarity}, {sentiment}')

        predicted_category = model.predict([article.text])[0]
        self.textboxes["Category"].setText(f'Category: {predicted_category}')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MainWindow()
    ex.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\Rizvi\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
